<div style="Tamplaged                                                                              ">
<b><center><font size="4">Practical Project 3</font></center></b>
<b><center><font size="3">Convolutional Neural Networks</font></center></b>
<b><center><font size="2">Predict Image</font></center></b>


</div>

**Notebook Developed by**: [Rene Jerez](https://github.com/renejerez)<br>
**email:**  rene.jerez@ubi.pt<br>

<h1>Classsification MNIST<span class="tocSkip"></span></h1>
<div class="toc">
  <ul class="toc-item">
    <li><span><a href="#Setup-and-Load-Data" data-toc-modified-id="Setup-and-Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup and Load Data</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Install-Libraries" data-toc-modified-id="Install-Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Install Libraries</a></span>
    </li>
    <li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import Libraries</a></span>
    </li>
    <li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load Data</a></span>
    </li>
    <li><span><a href="#Transformed-Data" data-toc-modified-id="Transformed-Data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Transformed Data</a></span>
    </li>
        </ul>
      </div>
    </li>
    <li><span><a href="Implement-a-classification-model" data-toc-modified-id="Implement-a-classification-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implement a classification model</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Architecture-based-on-the-doc-from-Tutorial/Resource" data-toc-modified-id="Architecture-based-on-the-doc-from-Tutorial/Resource-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Architecture based on the doc from Tutorial/Resource</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Architecture-based-on-the-doc-from-Tutorial/Resource" data-toc-modified-id="Architecture-based-on-the-doc-from-Tutorial/Resource-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Architecture based on the doc from Tutorial/Resource</a></span>
    </li>
    <li><span><a href="#Architecture-based-on-the-topology-recommended" data-toc-modified-id="Architecture-based-on-the-topology-recommended-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Architecture based on the topology recommended</a></span>
    </li>
    <li><span><a href="#Architecture-based-on-function-activate-LeakyReLU" data-toc-modified-id="Architecture-based-on-function-activate-LeakyReLU-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Architecture based on function activate LeakyReLU</a></span>
    </li>
        </ul>
      </div>
    </li>
    <li><span><a href="#Train-Networks" data-toc-modified-id="Train-Networks-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train Networks</a></span>
    </li>
        </ul>
      </div>
    </li>
    <li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Table-to-compare-the-6-models" data-toc-modified-id="Table-to-compare-the-6-models-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Table to compare the 6 models</a></span>
    </li>
    <li><span><a href="#Chart-to-see-the-6-model-with-Train-and-Test-data" data-toc-modified-id="Chart-to-see-the-6-model-with-Train-and-Test-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Chart to see the 6 model with Train and Test data</a></span>
    </li>
        </ul>
      </div>
    </li>
  </ul>
</div>

## Setup and Load Data

In [ ]:
##########################################################
# Contains Solutions and Notes based on Machine Learning Specialization by Andrew NG
##########################################################

### Install libraries

In [ ]:
#!pip install tensorflow
#!pip install torch
#!pip install torch torchvision
#!pip uninstall torch torchvision




### Import libraries

In [ ]:
##########################################################
# Import Libraries to start work
##########################################################
import gc
import os
import cv2
import shutil
import pickle
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from IPython.display import Image
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

import sklearn as sk
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

### PCA Resize by Hugo Proença script

In [ ]:
input_folder = r'C:\Users\rene_\Downloads\ML\week07\AR'
output_folder = r'C:\Users\rene_\Downloads\ML\week07\AR_out'

if os.path.isdir(output_folder):
    shutil.rmtree(output_folder)

os.mkdir(output_folder)

files = os.listdir(input_folder)

proportion = 0.05


In [ ]:

################################################################################################
#   STEP 1
#   Loading data and concatenate everything into a big matrix
################################################################################################

dataset = []
new_size = []

# Initialize a counter to track iterations
file_count = 0
total_files = sum(1 for f in files if f.find('.jpg') != -1)  # Count total valid '.jpg' files

for f in files:
    if f.find('.jpg') == -1:
        continue

    # Print the first file
    if file_count == 0:
        print(f"First file: {f}")
    
    # Process the image
    img = cv2.imread(os.path.join(input_folder, f))
    img = cv2.resize(img, None, fx=proportion, fy=proportion)
    new_size = img.shape
    dataset.append(img.flatten())

    # Print the last file
    if file_count == total_files - 1:
        print(f"Last file: {f}")

    file_count += 1

dataset = np.asarray(dataset)
df = pd.DataFrame(dataset)
df.to_csv('df.csv', index=False)

print('Dataset created: %d rows x %d columns' % (dataset.shape[0], dataset.shape[1]))


################################################################################################
#   STEP 2
#   Obtain mu, covariance matrix and eigen values/vectors.
################################################################################################

mu = np.mean(dataset, axis=0)

covar = np.cov(dataset.T)       #the "cov" function in numpy requires each instance in a different column

eig_values, eig_vectors = eigh(covar)



################################################################################################
#   STEP 3
#   Find the number of eigenvectors required to keep 99% of the information
################################################################################################

amount_information = 0.99

keep_inf = np.cumsum(np.flip(eig_values)) / np.sum(eig_values)  #This "eig" implementation returns the eigenvalues/eigenvectors
                                                                # in increasing order

pos = np.where(keep_inf >= amount_information)[0][0]

M = np.fliplr(eig_vectors[:, -pos:])                            #Select the top eigenvectors

################################################################################################
#   STEP 4
#   Write everything in file
################################################################################################

with open(os.path.join(output_folder, 'data.dat'), 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump([mu, covar, eig_vectors, eig_values, M, keep_inf], file)


################################################################################################
#   STEP 5
#   Write top-k eigenvectors
################################################################################################

k = 10
for i in range(k):
    img = np.reshape(M[:, i], new_size)
    img = ((cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX,
                                dtype=cv2.CV_32F)) * 255).astype(np.uint8)

    img = cv2.resize(img, None, fx=1.0/proportion, fy=1.0/proportion)
    cv2.imwrite(os.path.join(output_folder, 'eigenvector_%d.png' % i), img)


################################################################################################
#   STEP 6
#   Project images into PCA space and remap them into the original space
################################################################################################

for i, elt in enumerate(dataset):

    #project
    projected = np.matmul(elt - mu, M)

    #remap
    remapped = np.clip(np.matmul(projected, M.T) + mu, 0, 255)

    img = np.reshape(remapped, new_size)
    img = cv2.resize(img, None, fx=1.0 / proportion, fy=1.0 / proportion)
    cv2.imwrite(os.path.join(output_folder, files[i]), img)             # To perceive how much information is kept/lost


In [ ]:
################################################################################################
#   Validation process with multiple indices
################################################################################################

# Set a list of valid indices within the range of files
validation_indices = [3314, 1800]  # Example indices for validation

# Loop through each index in the validation list
for validation_index in validation_indices:
    print(f"\nValidating index: {validation_index}")

    # Load the original image directly
    original_image_path = os.path.join(input_folder, files[validation_index])
    original_image = cv2.imread(original_image_path, cv2.IMREAD_COLOR)

    # Check if the original image exists
    if original_image is None:
        print(f"Error: Original image not found or could not be loaded at index {validation_index}")
        continue  # Skip to the next index

    # Resized image (already in memory as part of the dataset)
    resized_image = np.reshape(dataset[validation_index], new_size).astype(np.uint8)

    # Projected image path
    projected_image_path = os.path.join(output_folder, files[validation_index])

    # Check if the projected image file exists
    if not os.path.exists(projected_image_path):
        print(f"Error: Projected image file not found at {projected_image_path}")
        continue  # Skip to the next index

    # Load the projected image
    projected_image = cv2.imread(projected_image_path, cv2.IMREAD_COLOR)

    # Validate that the projected image was loaded correctly
    if projected_image is None:
        print(f"Error: Projected image could not be loaded at {projected_image_path}")
        continue  # Skip to the next index

    # Convert BGR to RGB for matplotlib compatibility
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    resized_image_rgb = resized_image  # Already compatible
    projected_image_rgb = cv2.cvtColor(projected_image, cv2.COLOR_BGR2RGB)

    # Plot the images side by side
    plt.figure(figsize=(18, 9))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(original_image_rgb)
    plt.title(f"Original Image - Index {validation_index}")
    plt.axis("off")

    # Resized image
    plt.subplot(1, 3, 2)
    plt.imshow(resized_image_rgb)
    plt.title("Resized Image")
    plt.axis("off")

    # Projected image
    plt.subplot(1, 3, 3)
    plt.imshow(projected_image_rgb)
    plt.title("Projected Image")
    plt.axis("off")

    # Show the plot
    plt.tight_layout()
    plt.show()


## Results

### targets dataframe

In [ ]:
df['filename'] = files
df['gender'] = df['filename'].apply(lambda x: x.split('-')[0])  # Extract gender
df['id'] = df['filename'].apply(lambda x: x.split('-')[1])      # Extract ID
df['category'] = df['filename'].apply(lambda x: x.split('-')[2].split('.')[0])  # Extract category
df = df.drop(columns=['filename'], errors='ignore')
df.to_csv('df_target.csv', index=False)

In [ ]:
df_gender = pd.read_csv('df_target.csv', nrows=3000)
df_gender = df_gender.drop(columns=['filename', 'id', 'category'], errors='ignore')
print(df_gender['gender'].unique())  # Display unique values
df_gender['gender'] = df_gender['gender'].astype('category').cat.codes
print(df_gender['gender'].unique())  # Display unique values

### Resnet

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.image import resize

# Assuming `df` contains images flattened to 3306 columns and a label column
image_size = (57, 58)  # Original dimensions of the images
num_classes = 2  # Adjust to the number of classes in your dataset

# Separate features and labels
X = df_gender.iloc[:, :-1].values  # Assuming last column is the label
y = df_gender.iloc[:, -1].values  # Adjust as necessary for your dataset

# Reshape and preprocess the image data
X = X.reshape(-1, image_size[0], image_size[1], 1)  # Grayscale images (1 channel)
X_resized = resize(X, [224, 224])  # Resize to 224x224
X_rgb = np.repeat(X_resized, 3, axis=-1)  # Convert grayscale to RGB
X_rgb = X_rgb / 255.0  # Normalize the data

# Convert labels to one-hot encoding
y = to_categorical(y, num_classes)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_rgb, y, test_size=0.2, random_state=42)

# Load a pre-trained ResNet50 model with ImageNet weights
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, 
                    batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


In [ ]:
# Show the model summary
model.summary()


In [ ]:
import matplotlib.pyplot as plt

# Extract loss and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
df_model = pd.DataFrame(dataset)
df_model['filename'] = files  # Add the list of file names to the existing DataFrame
df_model['gender'] = df_model['filename'].str.split('-').str[0]
df_model['gender'] = df_model['gender'].astype('category').cat.codes
df_model = df_model.drop(columns=['filename'], errors='ignore')

In [ ]:
df_model.shape

In [ ]:
# Function to preprocess and predict for a single row
def predict_single_row(model, row, image_size=(57, 58), channels=1):
    # Extract features and reshape
    features = row[:-1].reshape(1, image_size[0], image_size[1], channels)
    # Normalize the data
    features = features / 255.0
    # Predict using the model
    prediction = model.predict(features)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction

In [ ]:
# List of indices to test till 3314 rows
test_indices = [1800, 3314]

# Loop through the selected indices
for index in test_indices:
    test_row = df_model.iloc[index].values  # Get the row by index
    
    # Extract the actual class (right class) from the last column
    actual_class = test_row[-1]

    # Predict on the test row
    predicted_class, probabilities = predict_single_row(model, test_row)

    # Format probabilities with class labels
    class_probabilities = {f"Class {i}": round(prob, 4) for i, prob in enumerate(probabilities[0])}

    # Display comparison
    print(f"Row Index: {index}")
    print(f"Actual Class: {actual_class}")
    print(f"Predicted Class: {predicted_class}")
    print(f"Class Probabilities: {class_probabilities}")
    print("-" * 50)  # Separator for readability
